In [2]:
import os
from datetime import datetime

import gradio as gr
import ipywidgets as widgets
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from RiskMetrics import RiskAnalysis, create_constraint, diversification_constraint
from scipy.optimize import minimize

In [3]:



# -------------------------
# Helper display function
# -------------------------
def display_scrollable_df(df, max_height="50vh", max_width="90vw"):
    style = f"""
    <div style="
        display: flex;
        justify-content: center;
        padding: 20px;
    ">
        <div style="
            overflow: auto;
            max-height: {max_height};
            max-width: {max_width};
            width: 100%;
            border: 1px solid #444;
            padding: 10px;
            background-color: #000;
            color: #eee;
            font-family: 'Arial Narrow', Arial, sans-serif;
            box-sizing: border-box;
        ">
            {df.to_html(classes='table', border=0, index=True)}
        </div>
    </div>
    """
    return HTML(style)


# -------------------------
# Placeholders for constraints
# -------------------------
def create_constraint(sign, limit, position):
    """Builds a simple single-asset constraint."""
    if sign == "≤":
        return [{"type": "ineq", "fun": lambda w, pos=position, lim=limit: lim - w[pos]}]
    elif sign == "≥":
        return [{"type": "ineq", "fun": lambda w, pos=position, lim=limit: w[pos] - lim}]
    else:  # equality
        return [{"type": "eq", "fun": lambda w, pos=position, lim=limit: w[pos] - lim}]


def diversification_constraint(sign, limit):
    """Example: limit on total number of non-zero weights."""
    # This is just a toy version: ensures sum(weights) respects limit
    if sign == "≤":
        return [{"type": "ineq", "fun": lambda w, lim=limit: lim - np.sum(w)}]
    elif sign == "≥":
        return [{"type": "ineq", "fun": lambda w, lim=limit: np.sum(w) - lim}]
    else:
        return [{"type": "eq", "fun": lambda w, lim=limit: np.sum(w) - lim}]


# -------------------------
# Main App
# -------------------------
def display_app(file):

    # === Utility functions (local scope) ===
    def sum_equal_one(weights):
        return np.sum(weights) - 1

    def objective(w):
        return np.sqrt(np.sum((w - w0) ** 2))

    def build_constraint(constraint_matrix):
        constraints = [{"type": "eq", "fun": sum_equal_one}]
        dico_map = {"=": "eq", "≥": "ineq", "≤": "ineq"}

        try:
            for row in range(constraint_matrix.shape[0]):
                temp = constraint_matrix[row, :]
                ticker = temp[0]

                if ticker not in drop_down_list:
                    continue

                sign = temp[1]
                limit = float(temp[2])

                if ticker == "All":
                    constraint = diversification_constraint(sign, limit)

                elif ticker in drop_down_list_asset:
                    position = np.where(full_matrix.index == ticker)[0][0]
                    constraint = create_constraint(sign, limit, position)

                elif ticker in drop_down_list_sector:
                    position = np.where(full_matrix.columns == ticker)[0][0]
                    if sign == "≤":
                        constraint = [{
                            "type": dico_map[sign],
                            "fun": lambda w, pos=position, lim=limit: lim - (w @ full_matrix_numpy)[pos],
                        }]
                    elif sign == "≥":
                        constraint = [{
                            "type": dico_map[sign],
                            "fun": lambda w, pos=position, lim=limit: (w @ full_matrix_numpy)[pos] - lim,
                        }]
                    else:
                        constraint = [{
                            "type": dico_map[sign],
                            "fun": lambda w, pos=position, lim=limit: (w @ full_matrix_numpy)[pos] - lim,
                        }]
                else:
                    constraint = []

                constraints.extend(constraint)

        except Exception as e:
            print(f"Error in build_constraint: {e}")

        return constraints

    # === Load Data ===
    data_file = file.parse(sheet_name=file.sheet_names)
    holdings = data_file["Holdings"].set_index("Name")
    holdings = holdings.loc[holdings.index != "Cash EUR"]
    holdings["Portfolio Weighting %"] /= holdings["Portfolio Weighting %"].sum()

    sheets = file.sheet_names.copy()
    sheets.remove("Holdings")

    transparency={}
    for sheet in sheets:
        temp = data_file[sheet].set_index("Name").iloc[:, 1:]
        temp = temp.loc[temp.index != "Cash EUR"]
        temp = temp.loc[holdings.index]
        transparency[sheet] = temp / 100

    full_matrix = pd.concat(transparency.values(), axis=1)
    full_matrix_numpy = full_matrix.to_numpy()
    w0 = holdings["Portfolio Weighting %"].loc[full_matrix.index].to_numpy()
    bounds_sectors={}
    for col in full_matrix.columns:
        min_bounds=round(full_matrix[col].min(),4)
        max_bounds=round(full_matrix[col].max(),4)
        name_max=full_matrix[col].idxmax()
        name_min=full_matrix[col].idxmin()

        bounds_sectors[col]=[min_bounds,max_bounds,name_min,name_max]

    bounds_sectors_dataframe=pd.DataFrame(bounds_sectors,index=['Lower Bound','Upper Bound','Name Min','Name Max']).T.round(4)

    
    drop_down_list_asset = list(full_matrix.index) + ["All"]
    drop_down_list_sector = list(full_matrix.columns)
    drop_down_list = drop_down_list_asset + drop_down_list_sector + [None]
    constraints_options = ["=", "≥", "≤"]

    # === Widgets ===
    dropdown1 = widgets.Dropdown(description="Assets:", value=None, options=drop_down_list)
    dropdown2 = widgets.Dropdown(description="Sign:", options=constraints_options)
    dropdown3 = widgets.FloatText(description="Limit")
    data = []

    def on_add_constraint_clicked(b):
        row = {"Asset": dropdown1.value, "Sign": dropdown2.value, "Limit": dropdown3.value}
        data.append(row)
        with constraint_output:
            constraint_output.clear_output()
            display(pd.DataFrame(data))

    add_constraint_btn = widgets.Button(description="Add Constraint", button_style="success")
    add_constraint_btn.on_click(on_add_constraint_clicked)

    constraint_output = widgets.Output()
    output = widgets.Output()

    def on_clear_clicked(b):
        data.clear()
        with constraint_output:
            constraint_output.clear_output()
            display(pd.DataFrame(columns=["Asset", "Sign", "Limit"]))

    clear_btn = widgets.Button(description="Clear All", button_style="danger")
    clear_btn.on_click(on_clear_clicked)

    def on_optimize_clicked(b):
        constraint_df = pd.DataFrame(data)
        constraints = build_constraint(constraint_df.to_numpy())
        bounds = [(0, 1) for _ in range(full_matrix.shape[0])]

        result = minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=constraints)
        opt_weights = result.x

        initial = pd.DataFrame(w0, index=full_matrix.index, columns=["Initial"])
        optimal = pd.DataFrame(opt_weights, index=full_matrix.index, columns=["Optimised"])
        res = pd.concat([initial, optimal], axis=1)

        change = res.copy()
        for col in change.columns:
            change[col] = res[col] - res["Initial"]

        sectors = (res.T @ full_matrix).T.round(4)
        change = (change.T @ full_matrix).T.round(4)

        def get_excel(b):
            with pd.ExcelWriter("Results.xlsx", engine="openpyxl") as writer:
                res.to_excel(writer, sheet_name="Weights")
                sectors.to_excel(writer, sheet_name="Sectors Allocation")
                change.to_excel(writer, sheet_name="Changes in Exposure")
                full_matrix.to_excel(writer, sheet_name="Transparency Matrix")
                bounds_sectors_dataframe.to_excel(writer, sheet_name="Sectors Bounds")

        bt_excel = widgets.Button(description="Get Excel")
        bt_excel.on_click(get_excel)

        with output:
            output.clear_output()
            display(display_scrollable_df(res.round(4)))
            display(display_scrollable_df(sectors.round(4)))
            display(display_scrollable_df(change.round(4)))
            display(bt_excel)

    optimize_btn = widgets.Button(description="Optimize Portfolio", button_style="primary")
    optimize_btn.on_click(on_optimize_clicked)

    constraint_ui = widgets.VBox(
        [
            widgets.VBox([dropdown1, dropdown2, dropdown3]),
            widgets.HBox([add_constraint_btn, clear_btn, optimize_btn]),
            constraint_output,
        ]
    )

    centered_constraint_ui = widgets.VBox(
        [constraint_ui, output],
        layout=widgets.Layout(
            display="flex",
            justify_content="center",
            align_items="center",
            width="auto",
            padding="10px",
        ),
    )

    tab_contents = ["Portfolio Optimization", "Transparency Matrix"]
    matrix_output = widgets.Output()
    with matrix_output:
        display(display_scrollable_df(full_matrix))
        display(display_scrollable_df(bounds_sectors_dataframe))

    children = [centered_constraint_ui, matrix_output]
    tab = widgets.Tab()
    tab.children = children

    for i, title in enumerate(tab_contents):
        tab.set_title(i, title)

    display(tab)


In [4]:
file=pd.ExcelFile('2025.03.28 ULGOALS Mstar Breakdown.xlsx')
display_app(file)